<a href="https://colab.research.google.com/github/ShauryaBhardwaj/Neural_Style_Transfer/blob/main/v1_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
dir_path = "/content/gdrive/MyDrive/NSTL/"

In [4]:
class VGG(nn.Module):
  def __init__(self):
    super(VGG , self).__init__()

    self.chosen_features = ['0','5','10','19','28']
    self.model = models.vgg19(pretrained = True).features[:29]

  def forward(self , x):
    features = []
    
    for layer_num , layer in enumerate(self.model):
      x = layer(x)

      if str(layer_num) in self.chosen_features:
        features.append(x)

      return features

In [5]:
def load_image(image_name):
  image = Image.open(image_name)
  image = loader(image).unsqueeze(0)
  return image.to(device)

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
image_size = 356

loader = transforms.Compose(
    [
     transforms.Resize((image_size , image_size)),
     transforms.ToTensor()
    ]
)

In [6]:
original_image = load_image(dir_path + 'original.jpg')
style_image = load_image(dir_path + 'style.jpg')

model = VGG().to(device).eval()
generated = original_image.clone().requires_grad_(True)

#Hyperparameters

total_steps = 6000
learning_rate = 0.001
alpha = 1
beta = 0.01
optimizer = optim.Adam([generated] , lr = learning_rate)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [9]:
for step in range(total_steps):
  generated_features = model(generated)
  original_features = model(original_image)
  style_features = model(style_image)

  style_loss , original_loss = 0 , 0

  for gen_feat , orig_feat , style_feat in zip(generated_features , original_features , style_features):
    batch_size , channel , height , width = gen_feat.shape
    
    original_loss += torch.mean((gen_feat - orig_feat)**2)

    G = gen_feat.view(channel , height*width).mm(gen_feat.view(channel , height*width).t())
    A = style_feat.view(channel , height*width).mm(style_feat.view(channel , height*width).t())

    style_loss += torch.mean((G - A) ** 2)

  total_loss = alpha*original_loss + beta*style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if step % 1000 == 0 :
    print(total_loss)
    save_image(generated , "generated.png")

tensor(2756762.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21795.8359, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6038.9800, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1602.7772, device='cuda:0', grad_fn=<AddBackward0>)
tensor(484.6345, device='cuda:0', grad_fn=<AddBackward0>)
tensor(114.8102, device='cuda:0', grad_fn=<AddBackward0>)
